<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Evaluation of Customer Reviews using Repetitions in Phoenix

This notebook walks through how to generate synthetic customer reviews, upload them into **Phoenix**, and run evaluations to identify patterns and repetitions.  
We’ll go step by step: generating data, structuring it into a dataset, and finally running experiments inside Phoenix to compare model outputs against reference labels.  
Along the way, we’ll also look at screenshots of the Phoenix UI to see how datasets and experiments are visualized.  


In the background, please set up a local instance of Phoenix. 
One way to do that is in your terminal, install arize-phoenix & run `phoenix serve`. For more information on other ways to run Phoenix locally, please check out our [documentation on self hosting](https://arize.com/docs/phoenix/self-hosting). 

### Setup & Installation
We start by installing the required dependencies:  
- **pandas** for data manipulation  
- **openai** for LLM calls  
- **arize-phoenix** to log and evaluate results

In [ ]:
%pip install pandas openai arize-phoenix

### Importing Libraries
Next, we import the libraries needed to:  
- Generate synthetic customer reviews using the OpenAI API  
- Register Phoenix for tracking experiments  
- Prepare data and send it into Phoenix for evaluation  


In [ ]:
import json
import os
import re
from getpass import getpass

import pandas as pd
from openai import AsyncOpenAI

from phoenix.client import AsyncClient
from phoenix.otel import register

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

openai_client = AsyncOpenAI()

client = AsyncClient()

tracer_provider = register(project_name="generating-datasets", auto_instrument=True)

### Generating Synthetic Customer Reviews
Here, we create a **few-shot prompt** that instructs the model to generate 25 product reviews for clothing items.  
This ensures we have a realistic dataset to evaluate with multiple tones and sentiments.  


In [ ]:
few_shot_prompt = """
You are a creative writer simulating customer product reviews for a clothing brand.
Generate exactly 25 unique reviews. Each review should be a few sentences long (max 200 words each) and sound like something a real customer might write.

Balance them across the following categories:
1. Highly Positive & Actionable → clear praise AND provides constructive suggestions for improvement.
2. Positive but Generic → generally favorable but vague.
3. Neutral / Mixed → highlights both pros and cons.
4. Negative but Actionable → critical but with constructive feedback.
5. Highly Negative & Non-Constructive → strongly negative, unhelpful venting.
6. Off-topic → not about clothing at all (e.g., a review mistakenly left about a different product or service). Don't say anything about how the product is not about clothing.

Constraints:
- Cover all 6 categories across the 25 reviews.
- Use a natural human voice, with realistic details.
- Constructive feedback should be specific and actionable.
- Make them really hard for someone else to classify. Add ambiguous reviews and reviews that are not clear, such as "The shirt is fine. Not bad, not great. Might buy again"
- Decide the classified label randomly first and then write the review. Double check all the reviews and make sure you classify them correctly.

OUTPUT SHAPE (JSON array ONLY; no extra text):
[
  {
    "input": str,
    "label": "highly positive & actionable" | "positive but generic" | "neutral" | "negative but actionable" | "highly negative" | "off-topic",
  }
]

Style Examples, Here are examples for guidance (do not repeat):
{
  "input": "I absolutely love the new denim jacket I purchased. The fit is perfect, the stitching feels durable, and I’ve already gotten compliments. The inside lining is soft and makes it comfortable to wear for hours. One small suggestion would be to add an inner pocket for a phone or keys — that would make it perfect. Overall, I’ll definitely be back for more.",
  "label": "highly positive & actionable"
}
{
  "input": "The T-shirt I bought was nice. The color was good and it felt comfortable. I liked it overall and would probably buy again.",
  "label": "positive but generic"
}
{
  "input": "The dress arrived on time and the material is soft. However, the sizing runs a bit small, and the shade of blue was lighter than pictured. It’s not bad, but I’m not as excited about it as I hoped.",
  "label": "neutral"
}
{
  "input": "The shoes looked stylish but the soles wore down quickly after just a month. If the company improved the durability of the soles, these would be a great buy. Right now, I don’t think they’re worth the price.",
  "label": "negative but actionable"
}
{
  "input": "This sweater is terrible. The worst thing I’ve ever bought. Waste of money.",
  "label": "highly negative & non-constructive"
}
{
  "input": "I'm very disappointed in my delivery. The dog food arrived late and was leaking.",
  "label": "off-topic"
}
"""

### Running the LLM to Generate Data
We send our prompt to the OpenAI model to generate the reviews.  
The output will be a structured set of text responses simulating customer feedback.  


In [ ]:
resp = await openai_client.chat.completions.create(
    model="gpt-5",
    messages=[{"role": "user", "content": few_shot_prompt}],
)
content = resp.choices[0].message.content.strip()

try:
    data = json.loads(content)
except json.JSONDecodeError:
    m = re.search(r"\[\s*{.*}\s*\]\s*$", content, re.S)
    assert m, "Model did not return a JSON array."
    data = json.loads(m.group(0))

### Creating a DataFrame
We load the generated responses into a **pandas DataFrame** with two columns:  
- `input`: the customer review text  
- `label`: the sentiment category we will later evaluate  


In [ ]:
df = pd.DataFrame(data)[["input", "label"]]
df

### Uploading to Phoenix
We now create a **Phoenix dataset** named `clothing-product-reviews` from our DataFrame. This allows us to track, explore, and evaluate the generated reviews inside Phoenix.  


In [ ]:
dataset_name = "my-customer-product-reviews"
dataset = await client.datasets.create_dataset(
    name=dataset_name,
    dataframe=df,
    input_keys=["input"],
    output_keys=["label"],
)
print("Dataset created.")

This is what your uploaded dataset will look like in the Phoenix UI! 

<img alt="uploaded dataset image" src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/phoenix-docs-images/repetitions_dataset_view.png" width="900"/>

### Defining the Evaluation Task
We define a task function that represents how we want to evaluate each review.  
This is where you could run another LLM pass (or a heuristic) to classify the review.  
Phoenix wraps each run into an **Example** object for easy logging.  


In [ ]:
async def my_task(theInput) -> str:
    TASK_PROMPT = f"""
        You will be given a single customer review about products from a clothing brand.
        Your job is to classify the type of review into a label.
        Please provide an explanation as to how you came to your answer.

        Allowed labels:
        - Highly Positive & Actionable
        - Positive but Generic
        - Neutral / Mixed
        - Negative but Actionable
        - Highly Negative & Non-Constructive
        - Off-topic

        Here is the customer review: {theInput}

        RESPONSE FORMAT:
        First provide your explanation, then on a new line write "LABEL:" followed by the exact label.
        Example:
        EXPLANATION: This review shows mixed sentiment with both positive and negative aspects...
        LABEL: Neutral / Mixed
    """
    resp = await openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": TASK_PROMPT}],
        temperature=1.0,
    )
    content = resp.choices[0].message.content.strip()

    if "LABEL:" in content:
        label = content.split("LABEL:")[-1].strip()
        return label
    else:
        return content.split("\n")[-1].strip()

### Running an Experiment
We run an experiment on our dataset using the defined task.  
This produces a labeled set of outputs that we can compare against our expectations.  
Phoenix records:  
- inputs (customer reviews)  
- outputs (model classifications)  
- metadata (timing, tokens, cost, etc.)  


In [ ]:
from phoenix.client.experiments import async_run_experiment

experiment = await async_run_experiment(
    dataset=dataset,
    task=my_task,
    experiment_name="testing explanations",
    client=client,
    repetitions=3,
)

This is what your uploaded experiment will look like in the Phoenix UI! You can click through the arrows as you want to look through each of the repetitions

<img alt="uploaded dataset image" src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/phoenix-docs-images/repetitions_experiment_view.png" width="900"/>

### Improving our Task
After analyzing our experiment results and the results of our repetitions, we can iterate on our eval template 
and see if there are any gaps in our prompt, or other things we may need to redefine. 

Using the same Task Prompt, I added clearer definitions for the labels as the biggest change to see if there's any improvement.

In [ ]:
async def improve_my_task(theInput) -> str:
    TASK_PROMPT = f"""
        You will be given a single customer review about products from a clothing brand, related to fashion clothing and apparel.
        Your job is to classify the type of review into a label.
        Please provide an explanation as to how you came to your answer.

        Allowed labels:
        - Highly Positive & Actionable: clear praise AND provides constructive suggestions for improvement.
        - Positive but Generic: generally favorable but vague.
        - Neutral / Mixed: highlights both pros and cons.
        - Negative but Actionable: critical but with constructive feedback.
        - Highly Negative & Non-Constructive: strongly negative, unhelpful venting.
        - Off-topic: not about clothing at all (e.g., a review mistakenly left about a different product or service).

        Here is the customer review: {theInput}

        RESPONSE FORMAT:
        First provide your explanation, then on a new line write "LABEL:" followed by the exact label.
        Example:
        EXPLANATION: This review shows mixed sentiment with both positive and negative aspects...
        LABEL: Neutral / Mixed
    """
    resp = await openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": TASK_PROMPT}],
        temperature=1.0,
    )
    content = resp.choices[0].message.content.strip()

    if "LABEL:" in content:
        label = content.split("LABEL:")[-1].strip()
        return label
    else:
        return content.split("\n")[-1].strip()

In [ ]:
from phoenix.client.experiments import async_run_experiment

experiment = await async_run_experiment(
    dataset=dataset,
    task=improve_my_task,
    experiment_name="improving my evaluation task",
    client=client,
    repetitions=3,
)